In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data1 = pd.read_csv('../prices/prices_round_2_day_-1.csv', header = 0, sep=";")
data2 = pd.read_csv('../prices/prices_round_2_day_0.csv', header = 0, sep=";")
data3 = pd.read_csv('../prices/prices_round_2_day_1.csv', header = 0, sep=";")

# Join rows of data1, data2 and data3
# data = pd.concat([data1, data2, data3])
data = data1
data.head()

,timestamp,ORCHIDS,TRANSPORT_FEES,EXPORT_TARIFF,IMPORT_TARIFF,SUNLIGHT,HUMIDITY,DAY
0,0,1200.00,1.5,10.5,-2.0,2500.0000,79.00000,-1
1,100,1201.75,1.5,9.5,-2.0,2499.4197,79.00410,-1
2,200,1201.75,1.5,9.5,-2.0,2498.8457,79.00821,-1
3,300,1201.75,1.5,9.5,-2.0,2498.2780,79.01234,-1
4,400,1201.75,1.5,9.5,-2.0,2497.7166,79.01649,-1


In [3]:

def create_X(df):
    sunlight_threshold = 7 * 60 * 6 # One unit is 10 minutes
    humidity_loss = np.maximum(0, np.maximum(60 - df['HUMIDITY'], df['HUMIDITY'] - 80))
    sunlight_loss = np.maximum(0, df['SUNLIGHT'] - sunlight_threshold)
    productivity_loss = np.minimum(4 * humidity_loss + (2 / 5) * sunlight_loss, 100)
    # df['productivity_loss'] = productivity_loss
    return pd.DataFrame({'transport': df['TRANSPORT_FEES'], 'export': df['EXPORT_TARIFF'], 'import': df['IMPORT_TARIFF'], 'loss': productivity_loss})

X = create_X(data).drop(index=0)
Y = data['ORCHIDS'].pct_change().drop(index=0)

X.head()

,transport,export,import,loss
1,1.5,9.5,-2.0,0.0
2,1.5,9.5,-2.0,0.0
3,1.5,9.5,-2.0,0.0
4,1.5,9.5,-2.0,0.0
5,1.5,9.5,-2.0,0.0


In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lr = LinearRegression()
lr.fit(X, Y)

# Coefficients
print("Coefficients:", lr.coef_)
print("Intercept:", lr.intercept_)

predictions = lr.predict(X)
mse = mean_squared_error(Y, predictions)
print("Mean squared error:", mse)
print("Mean Y squared change:", np.mean(Y ** 2))

Coefficients: [-6.92074933e-06  4.56749712e-06 -3.73457832e-06 -6.77912496e-08]
Intercept: -3.6811874719057004e-05
Mean squared error: 6.556156076190292e-07
Mean Y squared change: 6.556628977459695e-07
